In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import deque
from datetime import date
from glob import glob
from itertools import zip_longest
from os import system
from os.path import join
from time import sleep

import branca.colormap as cm
import gpxpy
import folium
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from folium.plugins import HeatMap
from scipy.linalg import norm
from selenium import webdriver
from tqdm.auto import tqdm, trange

In [ ]:
import sys

sys.path.insert(1, "../src/")
from mapping import load_gpx

In [ ]:
def chunks(iterable, chunk_size=3, overlap=0):
    # https://stackoverflow.com/questions/36586897/splitting-a-python-list-into-a-list-of-overlapping-chunks
    # we'll use a deque to hold the values because it automatically
    # discards any extraneous elements if it grows too large
    if chunk_size < 1:
        raise Exception("chunk size too small")
    if overlap >= chunk_size:
        raise Exception("overlap too large")
    queue = deque(maxlen=chunk_size)
    it = iter(iterable)
    i = 0
    try:
        # start by filling the queue with the first group
        for i in range(chunk_size):
            queue.append(next(it))
        while True:
            yield tuple(queue)
            # after yielding a chunk, get enough elements for the next chunk
            for i in range(chunk_size - overlap):
                queue.append(next(it))
    except StopIteration:
        # if the iterator is exhausted, yield any remaining elements
        i += overlap
        if i > 0:
            yield tuple(queue)[-i:]

## Load routes

In [ ]:
route_path = "../gpx/"

In [ ]:
data_files = sorted(glob(join(route_path, "*.gpx")))
data_files

## Find final bounds

In [ ]:
m = folium.Map(
    location=[51.236, -0.5644],
    zoom_start=14,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ",
)

In [ ]:
for idx, route_file in enumerate(data_files):
    route = load_gpx(route_file)
    folium.PolyLine(route, color="black", weight=1).add_to(m)

In [ ]:
final_bounds = m.get_bounds()

## Create screenshots

In [ ]:
output_path = "../screenshots/"

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("window-size=1280x720")  # optional

driver = webdriver.Chrome(
    executable_path="/usr/local/bin/chromedriver", options=options
)

In [ ]:
n_points_per_go = 10

In [ ]:
m = folium.Map(
    location=[51.236, -0.5644],
    zoom_start=14,
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ",
)

m.fit_bounds(final_bounds)

idx = 0
for route_file in tqdm(data_files):
    route = np.array(load_gpx(route_file))

    for segment in tqdm(list(chunks(route, n_points_per_go, 1))):
        folium.PolyLine(segment, color="black", weight=1).add_to(m)

        m.save("../screenshots/map.html")
        driver.get(
            "file:///Users/alvaroperez/Desktop/keep-exploring/screenshots/map.html"
        )
        sleep(0.5)
        driver.save_screenshot(
            f"/Users/alvaroperez/Desktop/keep-exploring/screenshots/{idx}.png"
        )
        idx += 1

driver.quit()

## Make video

In [ ]:
system(
    "ffmpeg -r 15 -i ../screenshots/%01d.png -c:v libx264 -preset veryslow -crf 0 ../screenshots/video.mp4"
)